Name: Stock Market Analysis

Project Type: Deep Learning: Long short-term memory (LSTM)

Analyzing the risk of a stock based on its previous performance history. Predicting future stock prices using LSTM.

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr

yf.pdr_override()

# For time stamps
from datetime import datetime


# The tech stocks we'll use for this analysis
tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN']

# Set up End and Start times for data grab
tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN']

end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)

for stock in tech_list:
    globals()[stock] = yf.download(stock, start, end)

company_list = [AAPL, GOOG, MSFT, AMZN] 
company_name = ["APPLE", "GOOGLE", "MICROSOFT", "AMAZON"]

for company, com_name in zip(company_list, company_name):
    company["company_name"] = com_name
    
df = pd.concat(company_list, axis=0)
df.tail(10)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,company_name
Date,,,,,,,
2023-01-19 00:00:00-05:00,94.739998,95.440002,92.860001,93.680000,93.680000,69002700,AMAZON
2023-01-20 00:00:00-05:00,93.860001,97.349998,93.199997,97.250000,97.250000,67307100,AMAZON
2023-01-23 00:00:00-05:00,97.559998,97.779999,95.860001,97.519997,97.519997,76501100,AMAZON
2023-01-24 00:00:00-05:00,96.930000,98.089996,96.000000,96.320000,96.320000,66929500,AMAZON
2023-01-25 00:00:00-05:00,92.559998,97.239998,91.519997,97.180000,97.180000,94261600,AMAZON
2023-01-26 00:00:00-05:00,98.239998,99.489998,96.919998,99.220001,99.220001,68523600,AMAZON
2023-01-27 00:00:00-05:00,99.529999,103.489998,99.529999,102.239998,102.239998,87678100,AMAZON
2023-01-30 00:00:00-05:00,101.089996,101.739998,99.010002,100.550003,100.550003,70691900,AMAZON
2023-01-31 00:00:00-05:00,101.160004,103.349998,101.139999,103.129997,103.129997,66527300,AMAZON
